In [1]:
from lxml import etree
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from matplotlib.patches import Polygon
import os
from bs4 import BeautifulSoup
import re
from collections import Counter
import re
import pandas as pd

In [3]:
path = '/home/noe/Documents/2022-2023/ENC/Mémoire/Guérin/Matenia/Noe/testSortie.xml'

In [4]:
tree = ET.parse(path)
root = tree.getroot()

In [5]:
with open(path,'r') as file:
    root = ET.parse(file)

In [6]:
path2 = '/home/noe/Documents/2022-2023/ENC/Mémoire/Guérin/XML/'
os.chdir(path2)

In [7]:
liste = os.listdir(path2)

In [8]:
liste = [s.replace('.xml', '') for s in liste]

In [9]:
liste = sorted(liste)

In [10]:
#dans le dossier XML il n'y a que les textes latins. Les ID sont justes, le nombre aussi. Il y a 357 textes lat

In [7]:
from bs4 import BeautifulSoup

In [13]:
with open(path,'r') as file:
    root = ET.parse(file)

In [20]:
from bs4 import BeautifulSoup

with open(path) as f:
    soup = BeautifulSoup(f,'xml')

textes = [tag.get_text() for tag in soup.find_all('body')]

print(textes)


['\n\n\n                        Philippus, et, universis et\n                        cetera Cordi habentes, ut condecet, inimicorum nequicias et impulsus totis conatibus repellendo,  quietem et pacem regni nostri et fidelium incolarum ejusdem, auxiliante Deo precipue et ipsorum mediante subditorum subsidio,\n                        procurare, ac providere ne mercatores et alii subditi nostri in suis mercaturis et negociis aliis impediri valeant vel exponere se\n                        periculis quibuscunque ; notum facimus\n                        quod nos, licet ad hoc\n                        personaliter intendere proponamus cucum viis et modis pluribus exquisitis, finaliter deliberato cumcum peritis in talibus etet quamplurimum famosis consilio, certam procedendi viam supper\n                        hoc\n                        providimus utilem et salubrem, quam\n                        dilectis et fidelibus\n                        nostris S. de Ruppe Choardi et J. Vigeri\n      

In [24]:
textes[0].replace("\n",'')

'                        Philippus, et, universis et                        cetera Cordi habentes, ut condecet, inimicorum nequicias et impulsus totis conatibus repellendo,  quietem et pacem regni nostri et fidelium incolarum ejusdem, auxiliante Deo precipue et ipsorum mediante subditorum subsidio,                        procurare, ac providere ne mercatores et alii subditi nostri in suis mercaturis et negociis aliis impediri valeant vel exponere se                        periculis quibuscunque ; notum facimus                        quod nos, licet ad hoc                        personaliter intendere proponamus cucum viis et modis pluribus exquisitis, finaliter deliberato cumcum peritis in talibus etet quamplurimum famosis consilio, certam procedendi viam supper                        hoc                        providimus utilem et salubrem, quam                        dilectis et fidelibus                        nostris S. de Ruppe Choardi et J. Vigeri                         militibu

In [16]:
#création d'un dictionnaire avec en clé le numéro du texte et le texte en valeur
#calcul de dates aussi

i = 1

dic = {}

while i <= 1757:
    
    with open(f'text_{i}.xml','r') as file:
        data = file.read()
    
    latin = []
    key_name = []
    
    for element in liste:
        if element in data:
            soup = BeautifulSoup(data,'xml')
            body_tag = soup.find('body')
            body_text = body_tag.text.strip()
            latin.append(body_text)
            key_name.append(element)
    
    dic.update({str(key_name):latin})
    
    i += 1

In [17]:
#enlever les valeurs vides du dictionnaire qui correspondent aux textes en français
keys_to_remove = []

for key, value in dic.items():
    if value == []:
        keys_to_remove.append(key)

for key in keys_to_remove:
    del dic[key]

In [18]:
#suppression des valeurs parasites qui ont plusieurs fois le même item dans la clé

keys_to_remove = []

for key, value in dic.items():
    if len(key)>15:
        keys_to_remove.append(key)

for key in keys_to_remove:
    del dic[key]

In [19]:
#Récupération des dates en supprimant les valeurs parasites aussi
keys = list(dic.keys())

In [20]:
tomes = []
for i in keys:
    tomes.extend(re.findall('[a-z]{4}[0-9]{1,2}_\d{4}',i))
num = []
for i in tomes:
    num.append(i.split('_')[1])

In [25]:
numéros = [s.lstrip('0') for s in num]
numéros

NameError: name 'num' is not defined

In [22]:
dates = []

for i in numéros:
    with open(f'text_{i}.xml','r') as file:
        data = file.read()
    soup = BeautifulSoup(data,'xml')
    date = soup.find('date')
    dates.append(str(date))

In [23]:
test = []
for i in range(len(dates)):
    if dates[i] == 'None':
        dates[i] = '0000'
    x = re.findall('\d{4}',dates[i])
    test.append(x)

In [24]:
final = []
for i in range(len(test)):
    final.append(test[i][0])

In [25]:
#Création du dataframe
colnames = ['id','text']
df = pd.DataFrame(dic.items())
df.columns = colnames
df['text'] = df['text'].apply(lambda x: str(x[0]))

In [26]:
def clean_text(string):
    first = re.sub("\n *","",string)
    return first

In [27]:
df['text'] = df['text'].apply(clean_text)

In [28]:
df['dates'] = final

In [29]:
def clean_string(s):
    s = s.replace("['", "")
    s = s.replace("']", "")
    return s

In [30]:
df['id'] = df['id'].apply(clean_string)

In [55]:
for i in range(len(df)):
    with open(f'{df["id"][i]}.txt','w') as file:
        file.write(df['text'][i])

In [49]:
df.to_excel('Guérin.xlsx')

In [62]:
liste_mots = df['text'].str.split().tolist()
x = set(liste_mots)

TypeError: unhashable type: 'list'

In [70]:
flat_list = [item for sublist in liste_mots for item in sublist]
x = list(set(flat_list))

In [73]:
matching_indexes = []

# Loop through each string in the list and check if it is the concatenation of two other strings in the list
for i, string in enumerate(x):
    for j in range(i+1, len(x)):
        if x[j] in string and x[i] in string:
            matching_indexes.append(i)
            matching_indexes.append(j)

In [75]:
len(matching_indexes)

388434

In [80]:
find_concatenated_words(x)

KeyboardInterrupt: 

In [79]:
def find_concatenated_words(words):
    word_set = set(words)
    result = []
    for i, word in enumerate(words):
        for j in range(i + 1, len(words)):
            if word + words[j] in word_set:
                result.append((i, j, words.index(word+words[j])))
            if words[j] + word in word_set:
                result.append((j, i, words.index(words[j]+word)))
    return result
